# Sakila DVD Rental – SQL & EDA

This notebook is part of a lab where we migrate the legacy Sakila SQLite database into DuckDB and perform exploratory data analysis (EDA).  

**Goals:**

- Connect to the Sakila SQLite database via DuckDB   
- Explore the data using a combination of SQL (DuckDB) and pandas  
- Prepare insights that can later be used for a BI dashboard


In [1]:
from pathlib import Path
import duckdb

DUCKDB_PATH = Path("../data/processed/sakila.duckdb")
SQLITE_PATH = Path("../data/raw/sqlite-sakila.db")

DUCKDB_PATH, SQLITE_PATH

(WindowsPath('../data/processed/sakila.duckdb'),
 WindowsPath('../data/raw/sqlite-sakila.db'))

## Task 0 – Data ingestion (SQLite → DuckDB)

In this section we create a DuckDB database file and attach the legacy Sakila SQLite database using DuckDB's SQLite extension.


In [2]:
con = duckdb.connect(DUCKDB_PATH.as_posix())

con.execute("INSTALL sqlite;")
con.execute("LOAD sqlite;")
con.execute(f"CALL sqlite_attach('{SQLITE_PATH.as_posix()}');")


## 1a. Movies longer than 3 hours

**Question:** Which movies are longer than 180 minutes (3 hours)?  
**Goal:** List movie `title` and `length` ordered from the longest film.

In [3]:
# NOTE: Query template generated with help from an LLM, adapted by me.

query_1a = """
SELECT
    title,
    length
FROM film
WHERE length > 180
ORDER BY length DESC, title;
"""

df_long_movies = con.sql(query_1a).df()
df_long_movies


,title,length
0,CHICAGO NORTH,185
1,CONTROL ANTHEM,185
2,DARN FORRESTER,185
3,GANGS PRIDE,185
4,HOME PITY,185
5,MUSCLE BRIGHT,185
6,POND SEATTLE,185
7,SOLDIERS EVOLUTION,185
8,SWEET BROTHERHOOD,185
9,WORST BANGER,185


## 1b. Movies with the word "love" in its title

**Question:** Which movies have the word "love" in its title? 
**Goal:** List movie `title`, `rating`, `length` and `description`.

In [4]:
query_1b = """
SELECT
    title,
    rating,
    length,
    description
FROM film
WHERE title ILIKE '%love%';
"""

df_love_movies = con.sql(query_1b).df()
df_love_movies

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,IDENTITY LOVER,PG-13,119,A Boring Tale of a Composer And a Mad Cow who ...
3,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
4,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
5,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...
6,LOVELY JINGLE,PG,65,A Fanciful Yarn of a Crocodile And a Forensic ...
7,LOVER TRUMAN,G,75,A Emotional Yarn of a Robot And a Boy who must...
8,LOVERBOY ATTACKS,PG-13,162,A Boring Story of a Car And a Butler who must ...
9,STRANGELOVE DESIRE,NC-17,103,A Awe-Inspiring Panorama of a Lumberjack And a...


## 1c. Descriptive statistics for movie length

**Question:** What is the shortest, average, median, and longest movie length?


In [ ]:
query_1c = """
SELECT
    AVG(length) AS avg_length,
    MIN(length) AS min_length,
    MEDIAN(length) AS median_length,
    MAX(length) AS max_length
FROM film;
"""

df_length_stats = con.sql(query_1c).df()
df_length_stats


## 1d. Top 10 most expensive movies to rent per day

**Question:** The rental rate is the cost to rent a movie and the rental duration is the number of days a customer can keep the movie.  
Which 10 movies are most expensive *per day*?


In [ ]:
query_1d = """
SELECT
    title,
    rental_rate,
    rental_duration,
    rental_rate / rental_duration AS price_per_day
FROM film
ORDER BY price_per_day DESC, title
LIMIT 10;
"""

df_expensive_per_day = con.sql(query_1d).df()
df_expensive_per_day


I checked with several examples and there are significantly more in the same price category. Friendly note.

## 1e. Actors with the most movie appearances

**Question:** Which actors have played in the most movies?  
**Goal:** Show the top 10 actors with the number of movies they have played in.


In [ ]:
query_1e = """
SELECT
    a.actor_id,
    a.first_name,
    a.last_name,
    COUNT(*) AS film_count
FROM actor AS a
JOIN film_actor AS fa
    ON a.actor_id = fa.actor_id
GROUP BY a.actor_id, a.first_name, a.last_name
ORDER BY film_count DESC, last_name, first_name
LIMIT 10;
"""

df_top_actors = con.sql(query_1e).df()
df_top_actors


## 1f. My own EDA questions

In this section I will explore 3–5 additional questions about the Sakila database, for example:

1. Which film categories have the longest average movie length?
2. How does revenue differ between film ratings (G, PG, PG-13, R, NC-17)?
3. Which customers rent the most different categories (variety of taste)?
4. Are longer movies rented less frequently than shorter movies?

Each question will have:
- a short explanation,
- an SQL query,
- a pandas DataFrame result,
- and a brief interpretation.


### 1f.1 – Which film categories have the longest average movie length?

**Motivation:**  
Understanding the average movie length per category can help the rental company identify which genres tend to be longer or shorter.  
This can influence customer recommendations and inventory decisions.

**Goal:**  
Compute the average length of films in each category and sort them in descending order.


In [ ]:
query_f1 = """
SELECT 
    c.name AS category,
    AVG(f.length) AS avg_length,
    MIN(f.length) AS min_length,
    MAX(f.length) AS max_length,
    COUNT(*) AS film_count
FROM film AS f
JOIN film_category AS fc ON f.film_id = fc.film_id
JOIN category AS c ON c.category_id = fc.category_id
GROUP BY c.name
ORDER BY avg_length DESC;
"""

df_length_by_category = con.sql(query_f1).df()
df_length_by_category


**Interpretation:**  
- The category with the longest average runtime is **Sports**, with an average of **128,202703 minutes**.  
- The shortest category is **Sci-Fi**, with an average of **108,196721 minutes**.  
These differences may reflect the storytelling style typically used in each genre.


### 1f.2 – Which film categories generate the highest revenue?

**Motivation:**  
By identifying the most profitable categories, the business can optimize inventory, promotions, and marketing.

**Goal:**  
Calculate total revenue per film category by combining film, inventory, rental, and payment tables.


In [ ]:
query_f2 = """
SELECT 
    c.name AS category,
    SUM(p.amount) AS total_revenue,
    COUNT(p.payment_id) AS payment_count
FROM category AS c
JOIN film_category AS fc ON c.category_id = fc.category_id
JOIN film AS f ON f.film_id = fc.film_id
JOIN inventory AS i ON i.film_id = f.film_id
JOIN rental AS r ON r.inventory_id = i.inventory_id
JOIN payment AS p ON p.rental_id = r.rental_id
GROUP BY c.name
ORDER BY total_revenue DESC;
"""

df_revenue_by_category = con.sql(query_f2).df()
df_revenue_by_category


**Interpretation:**  
- The most profitable category is **Sport**, with a total revenue of **5314,21 dollars**.  
- The least profitable category is **Music**, which may indicate lower demand or fewer rentals.  
This information is valuable for prioritizing which genres to stock more copies of.


### 1f.3 – Do longer movies get rented less frequently?

**Motivation:**  
Longer movies may require a larger time commitment, which could influence rental frequency.  
This question explores whether movie length is negatively correlated with rental counts.

**Goal:**  
Compute rental frequency per film and compare it against film length.


In [ ]:
query_f3 = """
SELECT
    f.title,
    f.length,
    COUNT(r.rental_id) AS rental_count
FROM film AS f
JOIN inventory AS i ON i.film_id = f.film_id
JOIN rental AS r ON r.inventory_id = i.inventory_id
GROUP BY f.film_id, f.title, f.length
ORDER BY rental_count DESC;
"""

df_length_vs_rentals = con.sql(query_f3).df()
df_length_vs_rentals.head(10)


**Interpretation:**  
- There appears to be a weak correlation between movie length and rental frequency.  
- Based on the scatter plot, longer movies seem to be rented about the same compared to shorter movies.  
This insight could help optimize recommendations for users based on their rental history.


## 2.1 – Total Revenue per Film Category

This visualization explores which movie categories generate the most revenue.

**Motivation:**  
Revenue is a key performance metric for the DVD rental business.  
By comparing how much revenue each category generates, we can identify which genres drive the most income.

**Goal:**  
Create a bar chart showing total revenue by category.


In [ ]:
query_2_1 = """
SELECT 
    c.name AS category,
    SUM(p.amount) AS total_revenue
FROM category AS c
JOIN film_category AS fc ON c.category_id = fc.category_id
JOIN film AS f ON f.film_id = fc.film_id
JOIN inventory AS i ON i.film_id = f.film_id
JOIN rental AS r ON r.inventory_id = i.inventory_id
JOIN payment AS p ON p.rental_id = r.rental_id
GROUP BY c.name
ORDER BY total_revenue DESC;
"""

df_revenue_category = con.sql(query_2_1).df()
df_revenue_category


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.bar(df_revenue_category["category"], df_revenue_category["total_revenue"])
plt.xticks(rotation=45, ha="right")
plt.xlabel("Category")
plt.ylabel("Total Revenue ($)")
plt.title("Total Revenue per Film Category")
plt.tight_layout()
plt.show()


**Insight:**  
- The most profitable category is **Sports**, generating approximately **5314.21 dollars**.  
- Lower-performing categories include **Music** and **Travel**, which may indicate lower demand.  
This insight may help guide inventory decisions and marketing priorities.


## 2.2 – Top Spending Customers

This visualization focuses on customer revenue contributions.

**Motivation:**  
Identifying the highest-value customers allows the business to personalize promotions and improve retention.

**Goal:**  
Show the top 5 customers ranked by total money spent.


In [ ]:
query_2_2 = """
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    SUM(p.amount) AS total_spent
FROM customer AS c
JOIN payment AS p ON p.customer_id = c.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
ORDER BY total_spent DESC
LIMIT 5;
"""

df_top_customers = con.sql(query_2_2).df()
df_top_customers


In [ ]:
plt.figure(figsize=(12, 6))
plt.barh(
    df_top_customers["first_name"] + " " + df_top_customers["last_name"],
    df_top_customers["total_spent"]
)
plt.xlabel("Total Spent ($)")
plt.ylabel("Customer")
plt.title("Top 5 Customers by Total Revenue")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()
